In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nitime

In [2]:
ts1 = np.loadtxt("ts1.txt")
ts2 = np.loadtxt("ts2.txt")
ts3 = np.loadtxt("ts3.txt")

In [3]:
import numpy as np
from Operations.CO_FirstCrossing import CO_FirstCrossing

def CO_glscf(y, alpha, beta, tau = 'tau'):
    """
    """
    # Set tau to first zero-crossing of the autocorrelation function with the input 'tau'
    if tau == 'tau':
        tau = CO_FirstCrossing(y, 'ac', 0, 'discrete')
    
    # Take magnitudes of time-delayed versions of the time series
    y1 = np.abs(y[:-tau])
    y2 = np.abs(y[tau:])


    p1 = np.mean(np.multiply((y1 ** alpha), (y2 ** beta)))
    p2 = np.multiply(np.mean(y1 ** alpha), np.mean(y2 ** beta))
    p3 = np.sqrt(np.mean(y1 ** (2*alpha)) - (np.mean(y1 ** alpha))**2)
    p4 = np.sqrt(np.mean(y2 ** (2*beta)) - (np.mean(y2 ** beta))**2)

    glscf = (p1 - p2) / (p3 * p4)

    return glscf    


In [11]:
CO_glscf(ts1, 0.9, 0.4, 3)

0.25284356429315435

In [13]:
import numpy as np
from Operations.CO_FirstCrossing import CO_FirstCrossing
from Operations.CO_glscf import CO_glscf

def CO_fzcglscf(y, alpha, beta, maxtau = None):
    """
    """
    N = len(y) # the length of the time series

    if maxtau is None:
        maxtau = N
    
    glscfs = np.zeros(maxtau)

    for i in range(1, maxtau+1):
        tau = i

        glscfs[i-1] = CO_glscf(y, alpha, beta, tau)
        if (i > 1) and (glscfs[i-1]*glscfs[i-2] < 0):
            # Draw a straight line between these two and look at where it hits zero
            out = i - 1 + glscfs[i-1]/(glscfs[i-1]-glscfs[i-2])
            return out
    
    return maxtau # if the function hasn't exited yet, set output to maxtau 


In [86]:
import warnings
import numpy as np

def DN_cv(x, k = 1):
    """
    Coefficient of variation

    Coefficient of variation of order k is sigma^k / mu^k (for sigma, standard
    deviation and mu, mean) of a data vector, x

    Parameters:
    ----------
    x (array-like): The input data vector
    k (int, optional): The order of coefficient of variation (k = 1 is default)

    Returns:
    -------
    float: The coefficient of variation of order k
    """
    if not isinstance(k, int) or k < 0:
        warnings.warn('k should probably be a positive integer')
        # Carry on with just this warning, though
    
    # Compute the coefficient of variation (of order k) of the data
    return (np.std(x, ddof=1) ** k) / (np.mean(x) ** k)


In [88]:
DN_cv(ts3)

26.269342687595913

In [113]:
DN_nlogL_norm(ts2)

1.2765413621752804

In [118]:
import numpy as np 

def DN_HighLowMu(y):
    """
    The highlowmu statistic.

    The highlowmu statistic is the ratio of the mean of the data that is above the
    (global) mean compared to the mean of the data that is below the global mean.

    Paramters:
    ----------
    y (array-like): The input data vector

    Returns:
    --------
    float: The highlowmu statistic.
    """
    mu = np.mean(y) # mean of data
    mhi = np.mean(y[y > mu]) # mean of data above the mean
    mlo = np.mean(y[y < mu]) # mean of data below the mean
    out = np.divide((mhi-mu), (mu-mlo)) # ratio of the differences

    return out


In [119]:
DN_HighLowMu(ts2)

1.0325203252032518

In [120]:
import numpy as np

def BF_SimpleBinner(xData, numBins):
    """
    Generate a histogram from equally spaced bins.
   
    Parameters:
    xData (array-like): A data vector.
    numBins (int): The number of bins.

    Returns:
    tuple: (N, binEdges)
        N (numpy.ndarray): The counts
        binEdges (numpy.ndarray): The extremities of the bins.
    """
    minX = np.min(xData)
    maxX = np.max(xData)
    
    # Linearly spaced bins:
    binEdges = np.linspace(minX, maxX, numBins + 1)
    N = np.zeros(numBins, dtype=int)
    
    for i in range(numBins):
        if i < numBins - 1:
            N[i] = np.sum((xData >= binEdges[i]) & (xData < binEdges[i+1]))
        else:
            # the final bin
            N[i] = np.sum((xData >= binEdges[i]) & (xData <= binEdges[i+1]))
    
    return N, binEdges


In [140]:
from scipy import stats

def DN_Cumulants(y, cumWhatMay = 'skew1'):
    
    """
    """

    if cumWhatMay == 'skew1':
        out = stats.skew(y)
    elif cumWhatMay == 'skew2':
        out = stats.skew(y, bias=False)
    elif cumWhatMay == 'kurt1':
        out = stats.kurtosis(y, fisher=False)
    elif cumWhatMay == 'kurt2':
        out = stats.kurtosis(y, bias=False, fisher=False)
    else:
        raise ValueError('Requested Unknown cumulant must be: skew1, skew2, kurt1, or kurt2')
    
    return out


In [141]:
DN_Cumulants(ts1, cumWhatMay='kurt1')

1.497362974297833

In [142]:
DN_Cumulants(ts1, cumWhatMay='kurt2')

1.4958467355321061

In [144]:
import numpy as np

def EN_ApEN(y, mnom = 1, rth = 0.2):
    """
    """
    r = rth * np.std(y, ddof=1) # threshold of similarity
    N = len(y) # time series length
    phi = np.zeros(2) # phi[0] = phi_m, phi[1] = phi_{m+1}

    for k in range(2):
        m = mnom+k # pattern length
        C = np.zeros(N - m + 1)
        # define the matrix x, containing subsequences of u
        x = np.zeros((N-m+1, m))

        # Form vector sequences x from the time series y
        x = np.array([y[i:i+m] for i in range(N - m + 1)])
        
        for i in range(N - m + 1):
            # Calculate the number of x[j] within r of x[i]
            d = np.abs(x - x[i])
            if m > 1:
                d = np.max(d, axis=1)
            C[i] = np.sum(d <= r) / (N - m + 1)

        phi[k] = np.mean(np.log(C))

    return phi[0] - phi[1]


In [149]:
EN_ApEN(ts3, 2, 0.4)

1.514020217676181

In [152]:
import matplotlib.mlab
import numpy as np
import scipy
from scipy import signal
import math
import matplotlib

def MD_hrv_classic(y):
    """
    """

    # Standard defaults
    y = np.array(y)
    diffy = np.diff(y)
    N = len(y)

    # ------------------------------------------------------------------------------
    # Calculate pNNx percentage
    # ------------------------------------------------------------------------------
    # pNNx: recommendation as per Mietus et. al. 2002, "The pNNx files: ...", Heart
    # strange to do this for a z-scored time series...
    Dy = np.abs(diffy)
    PNNxfn = lambda x : np.mean(Dy > x/1000)

    out = {}

    out['pnn5'] = PNNxfn(5) # 0.0055*sigma
    out['pnn10'] = PNNxfn(10) # 0.01*sigma
    out['pnn20'] = PNNxfn(20) # 0.02*sigma
    out['pnn30'] = PNNxfn(30) # 0.03*sigma
    out['pnn40'] = PNNxfn(40) # 0.04*sigma

    # ------------------------------------------------------------------------------
    # Calculate PSD
    # ------------------------------------------------------------------------------

    F, Pxx = signal.periodogram(y, window=np.hanning(N))

    # Calculate spectral measures such as subband spectral power percentage, LF/HF ratio etc.
    LF_lo = 0.04 # /pi -- fraction of total power (max F is pi)
    LF_hi = 0.15
    HF_lo = 0.15
    HF_hi = 0.4

    fbinsize = F[1] - F[0]

    # Calculate PSD
    f, Pxx = signal.periodogram(y, window='hann', detrend=False)
    f *= 2 * np.pi
    #print(Pxx)

    # Calculate spectral measures
    LF_lo, LF_hi = 0.04, 0.15
    HF_lo, HF_hi = 0.15, 0.4

    fbinsize = f[1] - f[0]
    indl = (f >= LF_lo) & (f <= LF_hi)
    indh = (f >= HF_lo) & (f <= HF_hi)
    indv = f <= LF_lo

    lfp = fbinsize * np.sum(Pxx[indl])
    hfp = fbinsize * np.sum(Pxx[indh])
    vlfp = fbinsize * np.sum(Pxx[indv])
    total = fbinsize * np.sum(Pxx)

    out['lfhf'] = lfp / hfp
    out['vlf'] = vlfp / total * 100
    out['lf'] = lfp / total * 100
    out['hf'] = hfp / total * 100

    # Triangular histogram index
    numBins = 10
    hist = np.histogram(y, bins=numBins)
    out['tri'] = len(y)/np.max(hist[0])

    # Poincare plot measures:
    # cf. "Do Existing Measures ... ", Brennan et. al. (2001), IEEE Trans Biomed Eng 48(11)
    rmssd = np.std(diffy, ddof=1)
    sigma = np.std(y, ddof=1)

    out["SD1"] = 1/math.sqrt(2) * rmssd * 1000
    out["SD2"] = math.sqrt(2 * sigma**2 - (1/2) * rmssd**2) * 1000

    return out


In [242]:
MD_hrv_classic(ts1)

{'pnn5': 0.982982982982983,
 'pnn10': 0.9669669669669669,
 'pnn20': 0.9359359359359359,
 'pnn30': 0.9029029029029029,
 'pnn40': 0.8708708708708709,
 'lfhf': 8.4832685576491e-08,
 'vlf': 1.1046723335508328e-08,
 'lf': 8.483267835924858e-06,
 'hf': 99.99999149238012,
 'tri': 4.830917874396135,
 'SD1': 99.72437694213063,
 'SD2': 996.9513129028788}

In [247]:
import numpy as np


def BF_MakeBuffer(y, bufferSize):
    """
    Make a buffered version of a time series.

    Parameters
    ----------
    y : array-like
        The input time series.
    buffer_size : int
        The length of each buffer segment.

    Returns
    -------
    y_buffer : ndarray
        2D array where each row is a segment of length `buffer_size` 
        corresponding to consecutive, non-overlapping segments of the input time series.
    """
    N = len(y)

    numBuffers = int(np.floor(N/bufferSize))

    # may need trimming
    y_buffer = y[:numBuffers*bufferSize]
    # then reshape
    y_buffer = y_buffer.reshape((numBuffers,bufferSize))

    return y_buffer


In [20]:
import numpy as np

def EN_wentropy(y, whaten = 'shannon', p = None):
    """
    Entropy of time series using wavelets.
    Uses a python port of the MATLAB wavelet toolbox wentropy function.

    Parameters:
    ----------
    y : array_like
        Input time series
    whaten : str, optional
        The entropy type:
        - 'shannon' (default)
        - 'logenergy'
        - 'threshold' (with a given threshold)
        - 'sure' (with a given parameter)
        (see the wentropy documentation for information)
    p : any, optional
        the additional parameter needed for threshold and sure entropies

    Returns:
    --------
    out : float
        Entropy value. 
    """
    N = len(y)

    if whaten == 'shannon':
        # compute Shannon entropy
        out = wentropy(y, 'shannon')/N
    elif whaten == 'logenergy':
        out = wentropy(y, 'logenergy')/N
    elif whaten == 'threshold':
        # check that p has been provided
        if p is not None:
            out = wentropy(y, 'threshold', p)/N
        else:
            raise ValueError("threshold requires an additional parameter, p.")
    elif whaten == 'sure':
        if p is not None:
            out = wentropy(y, 'sure', p)/N
        else:
            raise ValueError("sure requires an additional parameter, p.")
    else:
        raise ValueError(f"Unknown entropy type {whaten}")

    return out

# helper functions
# taken from https://github.com/fairscape/hctsa-py/blob/master/PeripheryFunctions/wentropy.py
def wentropy(x, entType = 'shannon', additionalParameter = None):

    if entType == 'shannon':
        x = np.power(x[ x != 0 ],2)
        return - np.sum(np.multiply(x,np.log(x)))

    elif entType == 'threshold':
        if additionalParameter is None or isinstance(additionalParameter, str):
            return None
        x = np.absolute(x)
        return np.sum((x > additionalParameter))

    elif entType == 'norm':
        if additionalParameter is None or isinstance(additionalParameter,str) or additionalParameter < 1:
            return None
        x = np.absolute(x)
        return np.sum(np.power(x, additionalParameter))

    elif entType == 'sure':
        if additionalParameter is None or isinstance(additionalParameter,str):
            return None

        N = len(x)
        x2 = np.square(x)
        t2 = additionalParameter**2
        xgt = np.sum((x2 > t2))
        xlt = N - xgt

        return N - (2*xlt) + (t2 *xgt) + np.sum(np.multiply(x2,(x2 <= t2)))

    elif entType == 'logenergy':
        x = np.square(x[x != 0])
        return np.sum(np.log(x))

    else:
        print("invalid entropy type")
        return None
    

In [22]:
EN_wentropy(ts2)

-0.22589264096854644